# MCP

<img src="images/student.png" width="150" alt="Several agents" style="float: left; margin-right: 15px; margin-bottom: 10px;"> Nous allons maintenant écrire une série d'agents pour conseiller les étudiants de l'IMT Atlantique dans leur chox de TAF. Nous allons définir des agents qui vont collecter des données sur les TAF disponibles, et dans un second temps répondre aux questions des étudiants pour positionner favorablement la TAF IoT vis à vis des autres.

Pour cela nous allons avoir besoin d'avoir des agents capables d'interagir avec l'extérieur pour aller chercher des informations sur le Web, ou même de stocker des informations pour assurer une cohérence entre deux interrogations. Nous avons vu pécédemment que l'on pouvait definir des outils, soit en utilisant le decorateur `@tool` qui va reprendre la fonction et utiliser la docstring pour comprendre le fonctionnement de la fonction et les arguments à lui fournir. 

Il est également possible dans l'API d'OpenAI de transformer un agent en outils avec la méthode `as_tool`.

Mais ici, nous allons utiliser des fonctions déjà packagées. La représentation de ces fonctions grâce à MCP (Model Context Protocol) est très populaire. Elle a été definie par Anthropic, la société qui fournit Claude. Il ne s'agit pas vraiment d'un protocole au sens réseaux, mais d'un habillage d'outils pour permettre une integration rapide dans un Agent.

MCP se decompose en deux élement un client et un serveur qui généralement tournent sur la même machine et utilisent les entrée/sorties standards (stdio) pour communiquer. Donc dans le code de l'agent, il faut:
* installer le serveur MCP
* lancer le serveur dans une co-routine
* donner à l'agent une description des outils et des formats utilisés par l'agent.

Un agent peut utiliser plusieurs serveur MCP, mais commençons par un exemple simple.

# Contexte

<img src="images/brain.png" width="150" alt="Several agents" style="float: left; margin-right: 15px; margin-bottom: 10px;"> Nous avions soulevé le problème la dernière fois avec la fenêtre de dialogue. Comme chaque requête est exécutée independamment de l'autre, le LLM ne se rappelle pas des informations que l'utilisateur lui a fournies. Une solution était de lui fournir l'historique du dialogue, mais les données ne sont pas structurées. 


Une solution consiste à enregistrer les relations entre élements apprises pendant le dialogue. Il existe un outil  `mcp-memory-libsql`. Comme son nom l'indique, l'outil est appelable par MCP, il va mémoriser des informations en utilisant une base de données SQL.


In [ ]:
import gradio
from dotenv import load_dotenv
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel
from openai import AsyncOpenAI
from agents.mcp import MCPServerStdio

from contextlib import AsyncExitStack

import asyncio
import os
import nest_asyncio
nest_asyncio.apply()  # Permet d'exécuter des boucles asynchrones

load_dotenv()  # Charger les variables d'environnement depuis le fichier .env

openai_model = OpenAIChatCompletionsModel(model="gpt-4.1-mini", openai_client=AsyncOpenAI())

RENNES_BASE_URL = "https://ragarenn.eskemm-numerique.fr/sso/ch@t/api"
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

# l'utilisation de os.environ permet de ne pas mettre les clés dans le code
# mais de les stocker dans un fichier .env qui n'est pas partagé et de generer une erreur si la clé n'est pas trouvée

rennes_client = AsyncOpenAI(base_url=RENNES_BASE_URL, api_key=os.environ["RENNES_API_KEY"])
rennes_model  = OpenAIChatCompletionsModel(model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", openai_client=rennes_client)

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=os.environ["GOOGLE_API_KEY"])
gemini_model  = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

ollama_client = AsyncOpenAI(base_url=OLLAMA_BASE_URL, api_key="dont matter")
ollama_model  = OpenAIChatCompletionsModel(model="gemma3:1b", openai_client=ollama_client)


memory_path = os.path.abspath(os.path.join(os.getcwd(), "memory/relations.db"))

MCP_params = [
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-memory"],"env":{"MEMORY_FILE_PATH": memory_path}},
    ]


In [5]:
import sqlite3
import json
from pathlib import Path

# Chemin vers la DB MCP
db_path = Path("/tmp/relations.db")
if not db_path.exists():
    print("Base de données MCP non trouvée")
else:
    # Dumper la DB
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    # Récupérer toutes les tables et leurs données
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = {table[0]: [dict(row) for row in cursor.execute(f"SELECT * FROM {table[0]}")] 
              for table in cursor.fetchall()}
    
    conn.close()
    
    # Afficher le résultat
    print(f"DB: {db_path}")
    for table, data in tables.items():
        print(f"{table}: {len(data)} lignes")
    
    # Les données sont dans la variable 'tables'
    print(json.dumps(tables, indent=2, default=str))

DatabaseError: file is not a database

Les premières lignes n'ont plus de secrets pour vous.

On crée plusieurs models vers plusieurs serveurs, vous pourrez choisir celui que vous préférez. Dans la suite, on va prendre OpenAI pour pouvoir tracer les requêtes, mais si vous êtes radin vous pouvez utilise Gemini, celui de Rennes ou même ollama à la place. Il faudra changer la variable dans le code quand le modèle sera défini.


Pour invoquer un serveur MCP, on doit definir une strucutre indiquant:
* comment installer le packetage:
    * `uvx` qui va retrouver les package python comme on l'a fait manuellement jusqu'ici,
    * `npx` qui va s'interesser aux packages javascript. Il faut installer `node`sur votre ordinateur dans une version supérieure à 18.
* les arguments durant l'installation, ici:
    * `-y` pour répondre oui à toutes les questions que l'installateur va poser, on a donc les choix par defaults
    *  `mcp-memory-libsql`que l'installateur va trouver ici https://www.npmjs.com/package/@modelcontextprotocol/server-memory
* les variables d'environnement:
    * ici le chemin où la base SQL va être stockée
    * mais on pourrait également trouver des Token d'API.

In [ ]:
async def chat_async(message, history):
    """Fonction async pour l'agent"""

    # with avec AsyncExitStack permet de fermer automatiquement les ressources quand on sort du bloc
    async with AsyncExitStack() as stack:
        # On prend le tableau d'arguments pour chaque serveur MCP et on crée une instance de MCPServerStdio pour chacun
        # le tableau mcp_servers contiendra les instances de MCPServerStdio
        # Ce code est très générique et permet d'ajouter ou de retirer des serveurs MCP en modifiant uniquement le tableau MCP_params

        mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in MCP_params]

        # ici one fait qu'afficher les outils disponibles et la description fournie au modèle.
        for server in mcp_servers:
            tools = await server.list_tools()  
            for tool in tools:
                print(f"Tool available: {tool.name} - {tool.description}")  

        return None


asyncio.run(chat_async("Hello", None))



On va définir les instructions système pour notre Agent, pour lui indiquer comment utiliser la base de données. 

In [ ]:
instructions = """
Tu es un assistant conversationnel avec une mémoire persistante parfaite.

PROTOCOLE DE MÉMOIRE OBLIGATOIRE :
1. TOUJOURS commencer par search_nodes pour récupérer les informations existantes
2. Quand l'utilisateur se présente, créer IMMÉDIATEMENT une entité avec create_entities
3. À CHAQUE nouveau fait important, l'ajouter avec create_entities ou mettre à jour
4. Créer des relations pertinentes avec create_relations

EXEMPLE D'UTILISATION DES OUTILS :
- Utilisateur dit "Je m'appelle Laurent" → create_entities avec name="Laurent", entityType="person", observations=["nom: Laurent"]
- Plus d'infos → ajouter à l'entité existante
- Toujours rechercher avant de répondre

IMPORTANT : Utilise SYSTÉMATIQUEMENT tes outils de mémoire pour CHAQUE conversation.
"""

async def chat_async(message, history):
    """Fonction async pour l'agent"""

    # with avec AsyncExitStack permet de fermer automatiquement les ressources quand on sort du bloc
    async with AsyncExitStack() as stack:
        # On prend le tableau d'arguments pour chaque serveur MCP et on crée une instance de MCPServerStdio pour chacun
        # le tableau mcp_servers contiendra les instances de MCPServerStdio
        # Ce code est très générique et permet d'ajouter ou de retirer des serveurs MCP en modifiant uniquement le tableau MCP_params

        mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in MCP_params]

        # ici one fait qu'afficher les outils disponibles et la description fournie au modèle.
        #for server in mcp_servers:
        #    tools = await server.list_tools()  
        #    for tool in tools:
        #        print(f"Tool available: {tool.name} - {tool.description}")  


        recruiting_agent = Agent(
            name="Recruiting Agent",
            instructions=instructions,
            model="gpt-4.1-mini",
            mcp_servers=mcp_servers
            )

        with trace("TAF LIST"):
            result = await Runner.run(
                recruiting_agent,
                message
            )

        return result.final_output

def chat_fn(message, history):
    """Fonction sync pour ChatInterface"""
    return asyncio.run(chat_async(message, history)) 

gradio.ChatInterface(chat_fn, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Error invoking MCP tool create_entities: ENOENT: no such file or directory, open '/Users/laurent/.npm/_npx/15b07286cbcc3329/node_modules/@modelcontextprotocol/server-memory/dist/memory/relations.db'
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/agents/mcp/util.py", line 104, in invoke_mcp_tool
    result = await server.call_tool(tool.name, json_data)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/agents/mcp/server.py", line 155, in call_tool
    return await self.session.call_tool(tool_name, arguments)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/mcp/client/session.py", line 264, in call_tool
    return await self.send_request(
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/mcp/shared/session.py", line 286, in send_request
    raise McpError(response_or_error.error)
mcp.shared.exceptions.Mcp

In [ ]:
from contextlib import AsyncExitStack
from dotenv import load_dotenv
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel
from openai import AsyncOpenAI
from agents.mcp import MCPServerStdio
from pydantic import BaseModel, Field
import os
from datetime import datetime
import asyncio
import nest_asyncio # for running asyncio in jupyter

nest_asyncio.apply() # necessary when asyncio is used into a jupyter notebook


load_dotenv(override=True)

RENNES_BASE_URL = "https://ragarenn.eskemm-numerique.fr/sso/ch@t/api"
rennes_client = AsyncOpenAI(base_url=RENNES_BASE_URL, api_key=os.environ["RENNES_API_KEY"])
rennes_model  = OpenAIChatCompletionsModel(model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", openai_client=rennes_client)

brave_env = {"BRAVE_API_KEY": os.environ["BRAVE_API_KEY"]}

Dans un premier temps nous allos utiliser deux serveurs MCP:
* server-filesystem qui va pouvoir écrire dans un répertoire que nous devons spécfier. c'est ce qui est fait au début du script. On rajoute `sandbox` à la fin du répertoire.
* brave search qui va interroger le moteur de recherche. Pour pourvoir y accéder il faudra une clé d'API que vous allez pouvoir trouver en créant un compte sur `https://brave.com/search/api/`

On crée ensuite une liste de serveurs MCP. Il existe deux méthodes principale pour les installer:
* uvx qui va retrouver les package python 
* npx qui va s'interesser aux packages javascript. Il faut installe `node`sur votre ordinateur dans une version supérieure à 18.

Ensuite on passe les arguments pour l'installation pour ces deux langages. On peut noter l'utilisation de la clé ènv`qui va permettre de passer des paramètres supplémentaires, comme la clé d'API pour Brave Search.

Les lignes de codes suivantes sont un peu plus absconces. On crée avec le `async with`une partie de code où plusiers co-routine vont pouvoir s'exécuter en parallèle:
* dans un premier temps on lance tous nos serveurs MCP, c'est à dire ceux qui ont été décrits dans le tableau `MCP_params`.
* le résultat est un tableau `mcp_servers`où tous les serveurs MCP sont décrits, 
* Ensuite on définit notre agent, et on l'execute en parallèle avec tous les serveurs MCP.
* Quand on quitte le block du `with`, toutes les connexions avec les serveurs sont fermées grace à la terminaison de `ÀsyncExitStack()`

In [ ]:
async def main():
    # Définition des paramètres des serveurs MCP
    sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))



    MCP_params = [
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}

    ]

    async with AsyncExitStack() as stack:
        mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in MCP_params]


        instructions = """
        You browse the internet to accomplish your instructions.
        You are highly capable at browsing the internet independently to accomplish your task, 
        including accepting all cookies and clicking 'not now' as
        appropriate to get to the content you need. If the site asks for authentication,
        validate, the credentials are already recorded.
        """

        try:
            # Utilisation des serveurs MCP dans le contexte de l'agent

            taf_list_agent = Agent(
                name="TAF List Agent",
                instructions=instructions,
                model="gpt-4.1-mini",
                mcp_servers=mcp_servers
            )

            with trace("TAF LIST"):
                result = await Runner.run(
                    taf_list_agent,
                    f"""
                    L'IMT Atlantique propose pour les étudiants de 2eme et 3eme année des parcours de formation
                    appelés TAF (Thématique d'Approfondissement de Formation).
                    1) Cherche sur le web avec Brave, les TAF proposées par l'IMT Atlantique.
                    2) construit leur nom sous la forme TAF xxxx, fait en une liste classée par ordre alphabetique
                    et stocke la dans le fichier TAF.md en y indiquant la date de mise à jour.
                    N'efface pas les données du fichier, ajoute juste les nouvelles TAF trouvées.
                    """
                )
                print(result.final_output)

        except Exception as e:
            print(f"Erreur: {e}")

        print ("End of agent")

if __name__ == "__main__":
    asyncio.run(main())

C'est bien le .md pour les IA, mais pour un programme python c'est plus compliqué à gérer. De plus on voit que la date mise dans le fichier est un peu fantaisiste. On va donc modifier les instructions pour forcer la réponse en JSON.

On va apporter d'autres modification:
* on va augmenter le nombre d'itération de l'agent à 50 car il n'est pas si simple de retrouver les TAF dans l'Internet.
* On va stocker le résultat au format JSON, notez que l'on indique la date actuelle dans la requête car les LLM ont une notion toute particulière du temps.
* On va créer un autre agent qui va être utiliser pour créer un descriptif plus détaillé des TAF que l'on a trouvé. Si le fichier `TAF.json`n'existe pas, alors notre agent principal va pouvoir appeler celui qui s'est déclaré pour reconstruire cette liste:
    * On ajoute la description `handoff_description` pour résumer ce que fait `TAF list agent`.
    * et dans la description de l'agent principal, on lui dit qu'il peut déléguer et on pointe sur la liste des agents secondaire.

In [ ]:
async def main():
    # Définition des paramètres des serveurs MCP
    sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))



    MCP_params = [
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}

    ]

    async with AsyncExitStack() as stack:
        mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in MCP_params]


        list_instructions = """
        You browse the internet to accomplish your instructions.
        You are highly capable at browsing the internet independently to accomplish your task, 
        including accepting all cookies and clicking 'not now' as
        appropriate to get to the content you need. If the site asks for authentication,
        validate, the credentials are already recorded.
        """

        taf_list_agent = Agent(
            name="TAF List Agent", 
            instructions=list_instructions, 
            model="gpt-4.1-mini",
            mcp_servers=mcp_servers,
            handoff_description="Use this agent to create or update the file TAF.json with the list of TAFs \
                offered by IMT Atlantique."        
            )
    
        try:
            # Utilisation des serveurs MCP dans le contexte de l'agent

            content_instructions="""
            You are an agent that browses the internet to find and retrieve information. You goal is to build a 
            precise description for a particular TAF, listed in the TAF.md file. If this file does not exist, or 
            its content is more than one day old, you have to call the TAF List Agent to create or update it.
            """
            taf_content_agent = Agent(
                name="TAF content Agent", 
                instructions=content_instructions, 
                model="gpt-4.1-mini",
                mcp_servers=mcp_servers,
                handoffs=[taf_list_agent]
            )

            with trace("TAF Content"):

                result = await Runner.run(
                    taf_content_agent,
                    f""" 
                    Aujourd'hui on est le {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.
                    L'IMT Atlantique propose pour les étudiants de 2eme et 3eme année des parcours de formation
                    appelés TAF (Thématique d'Approfondissement de Formation).

                    1) prend la liste des TAF contenu dans TAF.json, si le fichier n'existe pas ou est vieux d'un jour,
                    utilise l'agent TAF List Agent pour le créer ou le mettre à jour avec la liste des TAF trouvé par le
                    moteur de recherche.
                    2) pour chaque TAF contenue dans ce fichier, regarde si un fichier de description existe dans le répertoire.
                    3) si on n'a pas de description de la TAF, cherche sur internet pour avoir une description précise
                    du contenu avec un sylabus les plus détaillé possible, des métiers visés et de l'état du marché de
                    l'emploi dans ce domaine. Crée pour chaqu'un d'eux
                    un fichier spécifique dans le format markdown qui portera le nom de la TAF.

                    Limite ton nombre de requêtes à 5 par secondes.
                    """,
                    max_turns=50
                )
                print(result.final_output)

        except Exception as e:
            print(f"Erreur: {e}")

        print ("End of agent")

if __name__ == "__main__":
    asyncio.run(main())


Bon maintenant, on a rempli notre répertoire avec les informations concernant les TAF, mettez dedans PLIDO_BOOK_en.pdf

```cp PLIDO_BOOK_en.pdf sandbox/````

en on va pouvoir lancer un agent conversationnel, pour convaincre les étudiants de venir dans la TAF IoT. On reprend le code avec l'interface gradio que l'on avait développé avant, et l'on change les instructions.

In [ ]:
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display 


from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput

import gradio
import asyncio
import requests 


load_dotenv(override=True)

rennes_api_key = os.getenv("RENNES_API_KEY")
if not rennes_api_key:
    print("RENNES_API_KEY is missing")
    exit(1)

google_api_key = os.getenv("GOOGLE_API_KEY")
if not rennes_api_key:
    print("GOOLE_API_KEY is missing")
    exit(1)
    
RENNES_BASE_URL = "https://ragarenn.eskemm-numerique.fr/sso/ch@t/api"
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

rennes_client = AsyncOpenAI(base_url=RENNES_BASE_URL, api_key=rennes_api_key)
rennes_model  = OpenAIChatCompletionsModel(model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", openai_client=rennes_client)

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model  = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

ollama_client = AsyncOpenAI(base_url=OLLAMA_BASE_URL, api_key="dont matter")
ollama_model  = OpenAIChatCompletionsModel(model="gemma3:1b", openai_client=ollama_client)

recruit_instructions = """
You MUST use your tools systematically:
- ALWAYS start by checking memory for existing student information
- ALWAYS store new student information (name, interests, background) in memory
- ALWAYS read TAF documentation files when discussing program details
- ALWAYS search the web when you need current information

You are a recruitment agent. Your job is to convice students to apply to the TAF IoT. You have to
be polite, understand the questionning of the student, but you have to be persuasive and give them 
all the information they need to make a decision. You can use the document in the directory and
query the web to have more details information regarding the student's questions.

If the student asks for, you can provide a combination of two ATF he can do during the first 
or the second year. One of them has to be the TAF IoT

The TAF IoT is located in Rennes, you can add extra information to prove that Rennes is a dynamic city 
with a strong focus on technology and innovation. You can mention the presence of numerous tech companies, 
startups, and research institutions in the area, as well as the city's commitment to fostering innovation 
and supporting the growth of the digital economy. It is also a good place to party and have new friends.
"""

recruit_agent = Agent(name="recruit_agent", instructions=recruit_instructions, model=gemini_model)


async def chat_async(message, history):
    """Fonction async pour l'agent"""

    sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
    

    MCP_params = [
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-memory"],"env":{"MEMORY_FILE_PATH": "/tmp/memory.json"}},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": {"BRAVE_API_KEY": os.environ["BRAVE_API_KEY"]}}
    ]

    async with AsyncExitStack() as stack:
        mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in MCP_params]

        try:
            result = await Runner.run(recruit_agent, message)
            return result.final_output
        except Exception as e:
            return f"Erreur: {e}"

def chat_fn(message, history):
    """Fonction sync pour ChatInterface"""
    return asyncio.run(chat_async(message, history)) 

gradio.ChatInterface(chat_fn, type="messages").launch()

What we can see, it that it is impossible to have a dialog with the chat engine. There is no memory, so when you answer to a question he asked, the next round the agent has forgottent everything. 

One solution is to use a SQL database to memorize all the dialog.

In [ ]:
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display 


from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput

import gradio
import asyncio
import requests 


load_dotenv(override=True)

rennes_api_key = os.getenv("RENNES_API_KEY")
if not rennes_api_key:
    print("RENNES_API_KEY is missing")
    exit(1)

google_api_key = os.getenv("GOOGLE_API_KEY")
if not rennes_api_key:
    print("GOOLE_API_KEY is missing")
    exit(1)
    
RENNES_BASE_URL = "https://ragarenn.eskemm-numerique.fr/sso/ch@t/api"
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

rennes_client = AsyncOpenAI(base_url=RENNES_BASE_URL, api_key=rennes_api_key)
rennes_model  = OpenAIChatCompletionsModel(model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", openai_client=rennes_client)

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model  = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

ollama_client = AsyncOpenAI(base_url=OLLAMA_BASE_URL, api_key="dont matter")
ollama_model  = OpenAIChatCompletionsModel(model="gemma3:1b", openai_client=ollama_client)

recruit_instructions = """
You use your entity tools as a persistent memory to store and recall information about your conversations.
IMPORTANT: At the beginning of each conversation, you MUST:
1. First, use the memory tools to check if you have any previous information about this student
2. Store new information about the student (name, interests, preferences) in persistent memory
3. Read the TAF documentation files to have current information about the programs

You are a recruitment agent for the TAF IoT program. Your job is to convince students to apply.

WORKFLOW for each interaction:
1. Store/retrieve student information in memory
2. If student asks about TAF details, read the relevant markdown files in the directory
3. If you need current market information or specific details, search the web
4. Always save important student preferences and questions to memory for future reference

The TAF IoT is located in Rennes - mention the city's tech ecosystem and student life when relevant.
"""
Alternative : Instructions plus directives
Ou encore plus directif :
pythonrecruit_instructions = """
You MUST use your tools systematically:
- ALWAYS start by checking memory for existing student information
- ALWAYS store new student information (name, interests, background) in memory
- ALWAYS read TAF documentation files when discussing program details
- ALWAYS search the web when you need current information

You are a recruitment agent for the TAF IoT program...
[reste de vos instructions]
"""
Modification du code pour debugging
Ajoutez aussi cette ligne pour voir quels outils sont disponibles :
pythonrecruit_agent = Agent(
    name="recruit_agent", 
    instructions=recruit_instructions, 
    model=gemini_model,
    mcp_servers=mcp_servers,
    max_turns=10  # Limitez pour éviter les boucles infinies pendant les tests
)


async def chat_async(message, history):
    """Fonction async pour l'agent"""

    sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
    

    MCP_params = [
        {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/history.db"}},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
    ]

    async with AsyncExitStack() as stack:
        mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in MCP_params]

        recruit_agent = Agent(name="recruit_agent", 
                          instructions=recruit_instructions, 
                          model=gemini_model,
                          mcp_servers=mcp_servers)
        
        try:
            result = await Runner.run(recruit_agent, message)
            return result.final_output
        except Exception as e:
            return f"Erreur: {e}"

def chat_fn(message, history):
    """Fonction sync pour ChatInterface"""
    return asyncio.run(chat_async(message, history)) 

gradio.ChatInterface(chat_fn, type="messages").launch()